In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import pandas as pd
import pymupdf as pm

In [2]:
# %pip install PyMuPDF

In [103]:
# %pip install requests_html

In [124]:
webpage_num = [i for i in range(1, 40)]
url_page_list = [f"https://www.bok.or.kr/portal/singl/newsData/listCont.do?pageIndex={i}&targetDepth=3&menuNo=201154&syncMenuChekKey=2&depthSubMain=&subMainAt=&searchCnd=1&searchKwd=&depth2=200038&depth3=201154&depth4=200789" for i in webpage_num]
url = url_page_list[0]
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')
hrefs = soup.select("div.bd-line > ul > li.bbsRowCls")
href_list = [href.select('.title')[0].attrs.get('href') for href in hrefs]
"https://www.bok.or.kr"+href_list[0]

'https://www.bok.or.kr/portal/bbs/B0000245/view.do?nttId=10086296&searchCnd=1&searchKwd=&depth2=200038&depth3=201154&depth4=200789&depth=200789&pageUnit=10&pageIndex=1&programType=newsData&menuNo=200789&oldMenuNo=201154'

In [288]:
# 의사록 날짜 list
minute_date_list = []
for url_page in url_page_list:
    dateres = requests.get(url_page, 'html.parser')
    datesoup = BeautifulSoup(dateres.text)
    titlelist = datesoup.select('li.bbsRowCls > div.set')
    for title in titlelist:
        text_split = title.text.split('\n')
        text_strip = [i.strip() for i in text_split if i.strip()][0]
        minute_date = text_strip[text_strip.index(')') + 2:-1]
        minute_date_list.append(minute_date)

minute_date_list

['2024.7.18',
 '2024.7.11',
 '2024.6.26',
 '2024.6.13',
 '2024.5.23',
 '2024.5.9',
 '2024.4.12',
 '2024.3.28',
 '2024.3.14',
 '2024.2.22',
 '2024.2.8',
 '2024.1.25',
 '2024.1.11',
 '2023.12.28',
 '2023.12.14',
 '2023.11.30',
 '2023.10.19',
 '2023.9.26',
 '2023.9.14',
 '2023.8.24',
 '2023.7.27',
 '2023.7.13',
 '2023.6.21',
 '2023.6.8',
 '2023.5.25',
 '2023.4.11',
 '2023.3.23',
 '2023.3.9',
 '2023.2.23',
 '2023.1.13',
 '2022.12.22',
 '2022.12.8',
 '2022.11.24',
 '2022.11.10',
 '2022.10.27',
 '2022.10.12',
 '2022.9.22',
 '2022.9.8',
 '2022.8.25',
 '2022.7.28',
 '2022.7.13',
 '2022.6.22',
 '2022.6.9',
 '2022.5.26',
 '2022.4.14',
 '2022.3.24',
 '2022.3.10.',
 '2022.2.24.',
 '2022.1.14.',
 '2021.12.23.',
 '2021.12.9.',
 '2021.11.25.',
 '2021.10.12.',
 '2021.9.24.',
 '2021.9.9.',
 '2021.8.26.',
 '2021.7.29.',
 '2021.7.15.',
 '2021.6.30.',
 '2021.6.22.',
 '2021.6.10',
 '2021.5.27',
 '2021.4.15',
 '2021.3.25',
 '2021.3.11',
 '2021.2.25',
 '2021.1.28.',
 '2021.1.15.',
 '2020.12.24',
 '2020.12.10

In [289]:
len(minute_date_list)

390

In [132]:
# pdf 링크가 있는 사이트 추출(2024년 7월 18일 ~ 2005년 6월 9일)

link_list = []
webpage_num = [i for i in range(1,40)]
url_page_list = [f"https://www.bok.or.kr/portal/singl/newsData/listCont.do?pageIndex={i}&targetDepth=3&menuNo=201154&syncMenuChekKey=2&depthSubMain=&subMainAt=&searchCnd=1&searchKwd=&depth2=200038&depth3=201154&depth4=200789" for i in webpage_num]
for url in url_page_list:
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    hrefs = soup.select("div.bd-line > ul > li.bbsRowCls")
    href_list = [href.select('.title')[0].attrs.get('href') for href in hrefs]
    link_list.extend(["https://www.bok.or.kr"+sub_href for sub_href in href_list])

link_list

['https://www.bok.or.kr/portal/bbs/B0000245/view.do?nttId=10086296&searchCnd=1&searchKwd=&depth2=200038&depth3=201154&depth4=200789&depth=200789&pageUnit=10&pageIndex=1&programType=newsData&menuNo=200789&oldMenuNo=201154',
 'https://www.bok.or.kr/portal/bbs/B0000245/view.do?nttId=10086177&searchCnd=1&searchKwd=&depth2=200038&depth3=201154&depth4=200789&depth=200789&pageUnit=10&pageIndex=1&programType=newsData&menuNo=200789&oldMenuNo=201154',
 'https://www.bok.or.kr/portal/bbs/B0000245/view.do?nttId=10085855&searchCnd=1&searchKwd=&depth2=200038&depth3=201154&depth4=200789&depth=200789&pageUnit=10&pageIndex=1&programType=newsData&menuNo=200789&oldMenuNo=201154',
 'https://www.bok.or.kr/portal/bbs/B0000245/view.do?nttId=10085535&searchCnd=1&searchKwd=&depth2=200038&depth3=201154&depth4=200789&depth=200789&pageUnit=10&pageIndex=1&programType=newsData&menuNo=200789&oldMenuNo=201154',
 'https://www.bok.or.kr/portal/bbs/B0000245/view.do?nttId=10084604&searchCnd=1&searchKwd=&depth2=200038&dept

In [150]:
# pdf 링크 추출

pdf_link_list = []
for sub_link in link_list:
    res2 = requests.get(sub_link, 'html.parser')
    soup2 = BeautifulSoup(res2.text)
    sub_pdf_link = soup2.select_one(".viewer").attrs.get('href')
    pdf_link_list.append("https://www.bok.or.kr"+sub_pdf_link)

In [234]:
len(pdf_link_list)

390

In [152]:
pdf_test_link = pdf_link_list[0]

In [154]:
res3 = requests.get(pdf_test_link, 'html.parser')
soup3 = BeautifulSoup(res3.text)

In [169]:
pdf_test_link

'https://www.bok.or.kr/static/jslibrary/pdfjs/viewer.html?file=%2FfileSrc%2Fportal%2F8426eabe5ead44c4a1cfe6eccbc8ed3d%2F2%2F0cdeedba97704d7d93f46249561cc473.pdf'

In [168]:
import fitz

In [308]:
pdf_file_list = []
for link in link_list:
    res4 = requests.get(link, 'html.parser')
    soup4 = BeautifulSoup(res4.text)
    if soup4.select('.file')[0].attrs.get('href')[-3:] == 'hwp':
        pdf_file_link = soup4.select('.file')[1]
    else:
        pdf_file_link = soup4.select('.file')[0]
    pdf_file_list.append("https://www.bok.or.kr" + pdf_file_link.attrs.get('href'))

# for i in range(len(pdf_file_list)):
#     if pdf_file_list[i][-3:] == 'hwp':
#         pdf_file_list[i] = pdf_file_list[i].replace('hwp', 'pdf')
        
pdf_file_list

['https://www.bok.or.kr/fileSrc/portal/8426eabe5ead44c4a1cfe6eccbc8ed3d/2/0cdeedba97704d7d93f46249561cc473.pdf',
 'https://www.bok.or.kr/fileSrc/portal/4f0722fc879b4db098b3845c2e380955/2/dfe6a4ec98164e45ad0a777426015ea6.pdf',
 'https://www.bok.or.kr/fileSrc/portal/b13b402d895448968641c688803e26b6/2/46c845f0021643469862d1f198163915.pdf',
 'https://www.bok.or.kr/fileSrc/portal/4079947bb747468f8274e92123996b78/2/2ddb620bff684177b3466362e874a6e4.pdf',
 'https://www.bok.or.kr/fileSrc/portal/91734ac26098160ec0063d5dc26934bc/2/202406110257431401.pdf',
 'https://www.bok.or.kr/fileSrc/portal/c9eccabb0c3823e7831d574ab4d78ba9/2/202405280207257371.pdf',
 'https://www.bok.or.kr/fileSrc/portal/2862723d31283a5551de92155ad91596/2/202404300306223141.pdf',
 'https://www.bok.or.kr/fileSrc/portal/828d132d170200c1c098eb813ad6b720/2/202404121054534101.pdf',
 'https://www.bok.or.kr/fileSrc/portal/802a475bbb9c9b96684023cb25564216/2/202404010829075711.pdf',
 'https://www.bok.or.kr/fileSrc/portal/b3c1b8b75e5d38

In [309]:
len(pdf_file_list)

390

In [318]:
pdf_dict = {}
for minute_date, pdf_file in zip(minute_date_list, pdf_file_list):
    try:
        pdf_res = requests.get(pdf_file)
        pdf_data = fitz.io.BytesIO(pdf_res.content)
        doc = fitz.open(stream=pdf_data, filetype="pdf")
        full_text = ""
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)  # Load the page
            text = page.get_text("text")    # Extract text as a string
            full_text += text+"\n"
            
        keyword = "회의경과"
        keyword_index = full_text.index(keyword)
        full_text[keyword_index:]
        pdf_dict[minute_date] = full_text[keyword_index:]

    except Exception as e:
        # print('>>>', e)
        print(pdf_file)
        


In [322]:
pdf_dict.keys()

dict_keys(['2024.7.18', '2024.7.11', '2024.6.26', '2024.6.13', '2024.5.23', '2024.5.9', '2024.4.12', '2024.3.28', '2024.3.14', '2024.2.22', '2024.2.8', '2024.1.25', '2024.1.11', '2023.12.28', '2023.12.14', '2023.11.30', '2023.10.19', '2023.9.26', '2023.9.14', '2023.8.24', '2023.7.27', '2023.7.13', '2023.6.21', '2023.6.8', '2023.5.25', '2023.4.11', '2023.3.23', '2023.3.9', '2023.2.23', '2023.1.13', '2022.12.22', '2022.12.8', '2022.11.24', '2022.11.10', '2022.10.27', '2022.10.12', '2022.9.22', '2022.9.8', '2022.8.25', '2022.7.28', '2022.7.13', '2022.6.22', '2022.6.9', '2022.5.26', '2022.4.14', '2022.3.24', '2022.3.10.', '2022.2.24.', '2022.1.14.', '2021.12.23.', '2021.12.9.', '2021.11.25.', '2021.10.12.', '2021.9.24.', '2021.9.9.', '2021.8.26.', '2021.7.29.', '2021.7.15.', '2021.6.30.', '2021.6.22.', '2021.6.10', '2021.5.27', '2021.4.15', '2021.3.25', '2021.3.11', '2021.2.25', '2021.1.28.', '2021.1.15.', '2020.12.24', '2020.12.10', '2020.11.26', '2020.10.29', '2020.10.14', '2020.09.24', 

In [415]:
minute_df = pd.DataFrame(pdf_dict.values(), index=pdf_dict.keys())

In [416]:
minute_df.rename(columns={0:'content'}, inplace=True)

In [417]:
minute_df = minute_df.iloc[::-1]

In [418]:
idxlist = []
for item in minute_df.index:
    idxlist.append(item.replace('.','-'))


In [419]:
minute_df.index = idxlist

In [420]:
minute_df.reset_index(inplace=True)

In [421]:
idxlist

['2005-6-9',
 '2005-6-23',
 '2005-7-7',
 '2005-8-11',
 '2005-9-8',
 '2005-9-22',
 '2005-10-11',
 '2005-11-10',
 '2005-12-8',
 '2005-12-22',
 '2006-1-5',
 '2006-1-12',
 '2006-1-26',
 '2006-2-9',
 '2006-3-9',
 '2006-3-23',
 '2006-4-7',
 '2006-5-11',
 '2006-5-25',
 '2006-6-8',
 '2006-6-22',
 '2006-7-7',
 '2006-8-10',
 '2006-8-17',
 '2006-8-24',
 '2006-9-7',
 '2006-9-21',
 '2006-10-12',
 '2006-10-26',
 '2006-11-9',
 '2006-11-23',
 '2006-12-7',
 '2006-12-21',
 '2007-1-5',
 '2007-1-11',
 '2007-1-25',
 '2007-2-8',
 '2007-3-8',
 '2007-3-22',
 '2007-4-12',
 '2007-5-10',
 '2007-6-8',
 '2007-6-21',
 '2007-7-12',
 '2007-7-26',
 '2007-8-9',
 '2007-9-7',
 '2007-9-20',
 '2007-10-11',
 '2007-11-8',
 '2007-11-22',
 '2007-12-7',
 '2007-12-20',
 '2008-1-4',
 '2008-1-10',
 '2008-1-24',
 '2008-2-13',
 '2008-3-7',
 '2008-3-20',
 '2008-4-10',
 '2008-5-8',
 '2008-5-22',
 '2008-6-12',
 '2008-6-26',
 '2008-7-10',
 '2008-7-24',
 '2008-8-7',
 '2008-9-11',
 '2008-9-25',
 '2008-10-9',
 '2008-10-27',
 '2008-10-23',


In [429]:
x = idxlist[0].split('-')
for i in range(len(x)):
    if int(x[i]) < 10:
        x[i] = '0'+x[i]

''.join(x)

'20050609'

In [435]:
final_idxlist = []
for idx in idxlist:
    temp = idx.split('-')
    temp = [v for v in temp if v]
    for i in range(len(temp)):
        if int(temp[i])<10:
            temp[i] = '0'+temp[i]

    final_idxlist.append(''.join(temp))

final_idxlist

['20050609',
 '20050623',
 '20050707',
 '20050811',
 '20050908',
 '20050922',
 '20051011',
 '20051110',
 '20051208',
 '20051222',
 '20060105',
 '20060112',
 '20060126',
 '20060209',
 '20060309',
 '20060323',
 '20060407',
 '20060511',
 '20060525',
 '20060608',
 '20060622',
 '20060707',
 '20060810',
 '20060817',
 '20060824',
 '20060907',
 '20060921',
 '20061012',
 '20061026',
 '20061109',
 '20061123',
 '20061207',
 '20061221',
 '20070105',
 '20070111',
 '20070125',
 '20070208',
 '20070308',
 '20070322',
 '20070412',
 '20070510',
 '20070608',
 '20070621',
 '20070712',
 '20070726',
 '20070809',
 '20070907',
 '20070920',
 '20071011',
 '20071108',
 '20071122',
 '20071207',
 '20071220',
 '20080104',
 '20080110',
 '20080124',
 '20080213',
 '20080307',
 '20080320',
 '20080410',
 '20080508',
 '20080522',
 '20080612',
 '20080626',
 '20080710',
 '20080724',
 '20080807',
 '20080911',
 '20080925',
 '20081009',
 '20081027',
 '20081023',
 '20081107',
 '20081203',
 '20081211',
 '20081224',
 '20090102',

In [ ]:
minute_df['index'] = final_idxlist
minute_df.rename(columns={'index' : 'date'}, inplace=True)

In [443]:
minute_df

,date,content
0,20050609,회의경과\n〈의안 제21호 ― 통화정책방향〉\n(１) 전일 개최된 동향보고회의에서 ...
1,20050623,회의경과\n〈의안 제23호 ― 2005년 3/4분기 한국은행 총액한도대출의 한도 결...
2,20050707,회의경과\n〈의안 제25호 ― 통화정책방향〉\n(１) 전일 개최된 동향보고회의에서 ...
3,20050811,회의경과\n〈의안 제31호 ― 통화정책방향〉\n(１) 전일 개최된 동향보고회의에서 ...
4,20050908,회의경과\n〈의안 제32호 ― 통화정책방향〉\n(１) 전일 개최된 동향보고회의에서 ...
...,...,...
385,20240523,회의경과\n가. 의결안건\n〈의안제16호― 통화정책방향〉\n( 1 ) 전일개최된동향...
386,20240613,회의경과\n가. 의결안건\n<의안제17호- ｢한국은행의금융기관대출규정｣개정(안)>\...
387,20240626,회의경과\n가. 의결안건\n<의안제19호- ｢금융안정보고서(2024년6월)｣(안)>...
388,20240711,회의경과\n가. 의결안건\n〈의안제21호― 통화정책방향〉\n( 1 ) 전일개최된동향...


In [444]:
minute_df.to_csv("C:/Users/SesacPython/Desktop/workspace/Team2/hong/mpc_minute.csv")

In [454]:
announce_dates_raw = pd.read_csv("C:/Users/SesacPython/Downloads/기준금리날짜.csv")
full_announce_dates = announce_dates_raw.iloc[:206]

announce_dates = full_announce_dates.iloc[:]
announce_dates.announce_date

0      20050215
1      20050310
2      20050407
3      20050512
4      20050609
         ...   
201    20240111
202    20240222
203    20240412
204    20240523
205    20240711
Name: announce_date, Length: 206, dtype: int64

In [450]:
announce_dates.dtypes

announce_date    int64
start            int64
end              int64
dtype: object

In [455]:
x = list(minute_df['date'])
y = list(announce_dates.announce_date)

In [462]:
len(an_list)

36601